In [362]:
import graphlab as gl
import evaluate
import cPickle as pickle

In [363]:
# qword = pickle.load(open('../features/question_word_wordvec100.p', 'rb'))
# uword = pickle.load(open('../features/user_word_wordvec100.p', 'rb'))

In [483]:
ques = []
users = []
vals = []
with open('../train_data/invited_info_train.txt', 'r') as f1:
    for line in f1:
        qid, uid, val = line.rstrip('\n').split()
        ques.append(qid)
        users.append(uid)
        vals.append(int(val))

        

In [484]:
quesv = []
usersv = []
#vals = []
with open('../train_data/validate_nolabel.txt', 'r') as f1:
    line = f1.readline()
    for line in f1:
        qid, uid =  line.rstrip('\r\n').split(',')
        quesv.append(qid)
        usersv.append(uid)
        #vals.append(int(val))


In [485]:
quest = []
userst = []
#vals = []
with open('../train_data/test_nolabel.txt', 'r') as f1:
    line = f1.readline()
    for line in f1:
        qid, uid =  line.rstrip('\r\n').split(',')
        quest.append(qid)
        userst.append(uid)
        #vals.append(int(val))


In [486]:
sf = gl.SFrame({'user_id':users, 'item_id':ques, 'rating':vals})

In [487]:
sfv  = gl.SFrame({'user_id':usersv, 'item_id':quesv})

In [488]:
sft  = gl.SFrame({'user_id':userst, 'item_id':quest})

In [489]:
topics = []
with open('../train_data/user_info.txt', 'r') as f1:
    for line in f1:
        topic = map(str, (line.split()[1]).split('/'))
        topics.append(topic)

user_keys = pickle.load(open('../features/user_info_keys.dat', 'rb'))
ques_keys = pickle.load(open('../features/question_info_keys.dat', 'rb'))

#ufeat = []
#for i in range(len(topics)):
    #ufeat.append([1 if x in topics[i] else 0 for x in range(145)])
questopics = []
qnum = []

with open('../train_data/question_info.txt', 'r') as f1:
    for line in f1:
        questopics.append(line.split()[1])
        up, tans, ans = map(int, line.split()[4:7])
        qnum.append({'upvotes': up, 'topans':tans, 'answers':ans})

    

In [490]:
user_info = gl.SFrame({'user_id': user_keys, 'numeric_feature':uword})
ques_info = gl.SFrame({'item_id': ques_keys, 'numeric_feature':qword})
print ques_info

+-------------------------------+-------------------------------+
|            item_id            |        numeric_feature        |
+-------------------------------+-------------------------------+
| c1c0075239841777d5b01c40b3... | [-0.189298816425, -0.06583... |
| 367edcb36424493a7cf80f7090... | [-0.571765508888, 0.070196... |
| fb9e401d86f20205d97a25f3aa... | [-0.0909341260999, 0.44385... |
| e153659c6c654cd12122232fca... | [-0.158287365563, -0.32721... |
| c1718491292fd44b9b33397d5e... | [-0.0972034871296, 0.06211... |
| 85f22d70bd58c56708e9c3b276... | [0.164126001984, 0.1477459... |
| 3461879e8b67f1f9e08882f7f1... | [-0.219587751089, -0.96827... |
| 58695dc8b407850d8a028cec78... | [0.147465028776, -0.046943... |
| 39b402c90bda0ba4b5fe125ac8... | [0.0128764260132, -0.43330... |
| 8304fb2d94318d4768acf96acc... | [-0.200068923553, 0.062092... |
+-------------------------------+-------------------------------+
[8095 rows x 2 columns]
Note: Only the head of the SFrame is printed.
You ca

In [491]:
m1 = gl.ranking_factorization_recommender.create(sf, target='rating', ranking_regularization = 0.05, 
                                                 unobserved_rating_value=-0.5, verbose=False,
                                                 solver='auto')


Recsys training: model = ranking_factorization_recommender

In [492]:
scores = m1.predict(sfv)

In [493]:
predictions = []

#normalization
maxscore = max(scores)
minscore = min(scores)
for score in scores:
    predictions.append((score-minscore)/float(maxscore-minscore))

In [494]:
#fname = 'tempfile'
fname = '../validation/rankfactor.csv'
print len(quesv)
with open(fname, 'w') as f1:
    f1.write('qid,uid,label\n')
    for i in range(len(quesv)):
        f1.write(quesv[i]+','+usersv[i]+','+str(predictions[i])+'\n')
#print evaluate.ndcg(fname)
    

30466


In [495]:
scores = m1.predict(sft)

In [496]:
predictions = []

#normalization
maxscore = max(scores)
minscore = min(scores)
for score in scores:
    predictions.append((score-minscore)/float(maxscore-minscore))

In [497]:
fname = '../testsubmissions/rankfactor.csv'
print len(quest)
with open(fname, 'w') as f1:
    f1.write('qid,uid,label\n')
    for i in range(len(quest)):
        f1.write(quest[i]+','+userst[i]+','+str(predictions[i])+'\n')
#print evaluate.ndcg(fname)
    

30167
